In [1]:
import logging
import importlib
import TwitterFactCheck
import TwitterAPI
import pandas as pd
import altair as alt
import re
import spacy, en_core_web_sm, es_core_news_sm
import requests
import logging
import numpy as np
from typing import *
from datetime import *
import json
import traceback
importlib.reload(TwitterFactCheck)
importlib.reload(TwitterAPI)
logging.basicConfig(level=logging.INFO)

# 1. Manipulate original article and URLS

In [2]:
%%time
en_nlp = en_core_web_sm.load()
es_nlp = es_core_news_sm.load()
api = TwitterFactCheck.TwitterFactCheck("TwitterAPI.json")
pf_df = pd.read_csv("Data/PolitiFact.csv", sep="\t", index_col="CheckURL").dropna()
pf_df = pf_df[~pf_df["Rate"].isin(["True", "Mostly True"])].rename(columns={"Statement": "Headline"}).reset_index()
pf_df = pf_df[pf_df.iteration==pf_df.iteration.max()]
pf_df["TimeStamp"] = pd.to_datetime(pf_df["CheckTime"])
pf_df["Headline"] = pf_df["Headline"].apply(lambda x: " ".join(re.findall("\w*'?\w+", x)))
pf_df["EntityURL"] = pf_df["Sources"].apply(lambda x: [url[1:-1] for url in re.findall("\[.*?\]" , x) if len(url) > 3])
pf_df = pf_df[["Headline", "TimeStamp", "EntityURL"]].sort_values(["TimeStamp"])
uv_df = pd.read_csv("Data/Univision.csv", sep="\t", index_col="CheckURL").dropna()
uv_df["EntityURL"] = uv_df["EntityURL"].apply(lambda x: [url for url in x[1:-1].replace("'", "").split(", ") if url])
uv_df["TimeStamp"] = pd.to_datetime(uv_df["TimeStamp"])
uv_df = uv_df[["TimeStamp", "Headline", "EntityURL"]].sort_values(["TimeStamp"])
uv_df["Language"], pf_df["Language"] = "Es", "En"

CPU times: user 529 ms, sys: 39.6 ms, total: 569 ms
Wall time: 617 ms


In [3]:
%%time
def extract_entities(headline: str, model):
    headline = model(headline)
    entities = set(str(ent).lower() for ent in headline.ents)
    for token in headline:
        tag = token.tag_
        if (tag.startswith("NN") or tag.startswith("NOUN")) and not token.is_stop:
            lemma = token.lemma_.lower()
            valid = True
            for entity in entities:
                if set(entity.split(" ")) & set(lemma.split(" ")):
                    valid = False
                    break
            if valid:
                entities.add(lemma)
    return sorted({" ".join(re.findall("\w+", entity.lower())) for entity in entities})

pf_df["Entities"] = pf_df["Headline"].apply(lambda x: extract_entities(x, en_nlp))
uv_df["Entities"] = uv_df["Headline"].apply(lambda x: extract_entities(x, es_nlp))
# df = pd.concat([pf_df, uv_df])
df = pd.concat([pf_df])
df.head(3)

CPU times: user 1.19 s, sys: 0 ns, total: 1.19 s
Wall time: 1.2 s


,Headline,TimeStamp,EntityURL,Language,Entities
573,In Texas because of the laws they passed a 95 ...,2022-02-03,[https://transcripts.cnn.com/show/cnnt/date/20...,En,"[1950, 95 year old, law, mail, number, registr..."
577,On eliminating the filibuster,2022-02-04,[https://www.baldwin.senate.gov/in-the-news/ta...,En,[filibuster]
575,Few If Any Doses of Spikevax or Comirnaty Avai...,2022-02-04,"[https://archive.is/Ry0Xh, https://www.usatoda...",En,"[comirnaty, doses, fda approvals, spikevax, un..."


# 2. Extract tweets based on URL link

In [24]:
true_domain

{'northernvatimes.com',
 'mypalmbeachpost.com',
 'rockdalecitizen.com',
 'thenewsliteracyproject.org',
 'politicalwire.com',
 'homertribune.com',
 'smithmountaineagle.com',
 'bellingcat.com',
 'interiowo.pl',
 'wkar.org',
 'marshallnews.com',
 'brookskraft.com',
 'eastoregonian.com',
 'borgernewsherald.com',
 'matadornetwork.com',
 'keyetv.com',
 'georgetowner.com',
 'heraldnewstn.com',
 'ourquadcities.com',
 'upnorthlive.com',
 'ipress.com.hk',
 'yaf.org',
 'thepostnewspaper.net',
 'tuftsobserver.org',
 'forwardprogressives.com',
 'bizwest.com',
 'hawaiiarmyweekly.com',
 'nipawinjournal.com',
 'forecastadvisor.com',
 'jax-cdn.com',
 'keithbush.com',
 'modbee.com',
 'equities.com',
 'cbs12.com',
 'saintpetersblog.com',
 'ilrt.org',
 'groesbeckjournal.com',
 'campbellrivermirror.com',
 'bizmonthly.com',
 'spotlightheroes.com',
 'theguardianonline.com',
 'macleans.ca',
 'usanewspapers.com',
 'mypeoplepc.com',
 'mndaily.com',
 'north99.org',
 'nbr.co.nz',
 'danielbrett.com',
 'bonnyvillen

In [24]:
df_domain = pd.read_csv("Data/WebsiteCredibility.csv", sep="\t")
fake_domain = set(df_domain[(df_domain.fake==1)|(df_domain.lowcred==1)].domain)

with open("Data/UnrelatedWebsites.json", "r") as f:
    true_domain = set(json.loads(f.read()))
    true_domain = true_domain | set(df_domain[df_domain.traditional==1].domain) 
def check_domain(keyword):
    for domain in true_domain:
        if domain.lower() in keyword:
            return 2
        
    for domain in fake_domain:
        if domain.lower() in keyword:
            return 0
    
    return 1

In [4]:
%%time 
def extract_tweet(fact_check, day_range: int=7, threshold: int=5000):
    start_time = fact_check.TimeStamp - timedelta(days=day_range)
    end_time = min(fact_check.TimeStamp + timedelta(days=day_range), pd.datetime.now())
    columns = ['Keyword', 'source', 'created_at', 'author_id', 'text','public_metrics', 'entities', 'Method', 'Headline', 'Language']
    
    def large_query_res(counts):
        large_query_df = pd.DataFrame()
        large_query_df.loc[0, columns] = np.NaN
        large_query_df["source"] = "Large Query" if counts else "Empty Query"
        large_query_df["id"] = counts
        return large_query_df

    # Search for headline
    def headline_query(headline):
        headline = re.sub("and|not|or", "", headline.lower())
        return " ".join(re.findall("\w+", headline))
    query = headline_query(fact_check.Headline)
    headline_counts = api.count_tweets(query, start_time=start_time, end_time=end_time)["tweet_count"].sum()
    if headline_counts > threshold:
        headline_tweets = large_query_res(headline_counts)
    else:
        headline_tweets = api.search_tweets(query, start_time=start_time, end_time=end_time)
    headline_tweets["Method"], headline_tweets["Keyword"] = "Headline", query
    # Search for entities
    def entity_query(entities):
        entity_query = " ".join(entities)
        return re.sub("and|not|or", "", entity_query.lower())
    query = entity_query(fact_check.Entities)
    entity_counts = api.count_tweets(query, start_time=start_time, end_time=end_time)["tweet_count"].sum() if fact_check.Entities else 0
    if entity_counts > threshold or entity_counts == 0:
        entity_tweets = large_query_res(entity_counts)
    else:
        entity_tweets = api.search_tweets(query, start_time=start_time, end_time=end_time)
    entity_tweets["Method"], entity_tweets["Keyword"] = "Entity", " ".join(fact_check.Entities)
    # Search for URL
    url_tweets = list()
    sus_url = None
    for url in fact_check.EntityURL:
        if check_domain(url) < 2:
            sus_url = url
            break
    if not sus_url is None:
        url_query = 'url:"{}"'.format(sus_url)
        url_counts = api.count_tweets(url_query, start_time=start_time, end_time=end_time)["tweet_count"].sum()
        if url_counts > threshold:
            url_df = large_query_res(url_counts)
            url_tweets.append(url_df)
        else:
            url_df = api.search_tweets(url_query, start_time=start_time, end_time=end_time)
            url_df["Keyword"] = url
        url_tweets.append(url_df)
    url_tweets = pd.concat(url_tweets)
    url_tweets["Method"] = "URL"

    df = pd.concat([headline_tweets, entity_tweets, url_tweets]).reset_index(drop=True)
    df["Headline"], df["Language"] = fact_check.Headline, fact_check["Language"]
    return df

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.29 µs


In [7]:
res = list()
err = list()
huge_err = dict()
for i in range(len(df)):
    try:
        res.append(extract_tweet(df.iloc[i]))
    except Exception:
        print("{} error".format(i))
        err.append(i)
    if i % 50 == 49:
        try:
            res = pd.concat(res)
            res.text = res.text.replace("\s+", " ")
            res["content"] = res.text.str.replace("(@[\w|\d]+|https\S+)|\,|!|\?|\W|^RT ", " ").str.replace("\s{2,}", "").str.replace("^\s+\$", "")
            res.to_csv("Data/tmp/{}_search.csv".format(i), sep="\t")
            res = list()
        except Exception:
            huge_errs[i] = traceback.format_exc()
            
try:
    res = pd.concat(res)
    res.text = res.text.replace("\s+", " ")
    res["content"] = res.text.str.replace("(@[\w|\d]+|\#[\w|\d]+|https\S+)|\,|!|\?|\W|^RT ", " ").str.replace("\s{2,}", "").str.replace("^\s+\$", "")
    res.to_csv("Data/tmp/{}_search.csv".format(i), sep="\t")
except Exception:
    huge_err[i] = traceback.format_exc()

<timed exec>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 17 tweets in total
INFO:TwitterAPI Logger:Query 4 tweets in total
INFO:TwitterAPI Logger:Query 1 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 3 tweets in total
INFO:TwitterAPI Logger:Query 3 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
<timed exec>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 5 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 782 tweets in total
INFO:TwitterAPI Logger:Query 1 tweets in total
INFO:TwitterAPI L

INFO:TwitterAPI Logger:Query 139 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 24 tweets in total
INFO:TwitterAPI Logger:Query 10 tweets in total
<timed exec>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
INFO:TwitterAPI Logger:Query 6 tweets in total
INFO:TwitterAPI Logger:Query 21 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 12 tweets in total
INFO:TwitterAPI Logger:Query 1 tweets in total
INFO:TwitterAPI Logger:Query 121 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
<timed exec>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
INFO:TwitterAPI Logger:Query 3533 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:Twit

INFO:TwitterAPI Logger:Query 22 tweets in total
INFO:TwitterAPI Logger:Query 38 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 1 tweets in total
INFO:TwitterAPI Logger:Query 14 tweets in total
INFO:TwitterAPI Logger:Query 133 tweets in total
INFO:TwitterAPI Logger:Query 6 tweets in total
<timed exec>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 11 tweets in total
INFO:TwitterAPI Logger:Query 103 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 6 tweets in total
INFO:TwitterAPI Logger:Query 11 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 23 tweets in total
INFO:TwitterAPI Logger:Query 4

INFO:TwitterAPI Logger:Query 2 tweets in total
<timed exec>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 18 tweets in total
INFO:TwitterAPI Logger:Query 111 tweets in total
INFO:TwitterAPI Logger:Query 13 tweets in total
<timed exec>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
INFO:TwitterAPI Logger:Query 1 tweets in total
INFO:TwitterAPI Logger:Query 2 tweets in total
INFO:TwitterAPI Logger:Query 1 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 8 tweets in total
<timed exec>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
INFO:TwitterAPI Logger:Query 0 tweets in to

INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
<timed exec>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 1 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 2 tweets in total
INFO:TwitterAPI Logger:Query 88 tweets in total
INFO:TwitterAPI Logger:Query 3 tweets in total
INFO:TwitterAPI Logger:Query 1 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 64 tweets in total
INFO:TwitterAPI Logger:Query 82 tweets in total
INFO:TwitterAPI Logger:Query 446 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 1 tweets in total
INFO:TwitterAPI Logger:Query 55 twe

...

INFO:TwitterAPI Logger:Query 11 tweets in total
INFO:TwitterAPI Logger:Query 617 tweets in total
<timed exec>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 4 tweets in total
INFO:TwitterAPI Logger:Query 144 tweets in total
INFO:TwitterAPI Logger:Query 8 tweets in total
INFO:TwitterAPI Logger:Query 14 tweets in total
INFO:TwitterAPI Logger:Query 4 tweets in total
INFO:TwitterAPI Logger:Query 5 tweets in total
INFO:TwitterAPI Logger:Query 43 tweets in total
INFO:TwitterAPI Logger:Query 36 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
<timed exec>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 43 tweets in total
INFO:Twitte

INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 5 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 3 tweets in total
INFO:TwitterAPI Logger:Query 5 tweets in total
<timed exec>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
INFO:TwitterAPI Logger:Query 144 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 15 tweets in total
INFO:TwitterAPI Logger:Query 3 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 1 tweets

INFO:TwitterAPI Logger:Query 2 tweets in total
INFO:TwitterAPI Logger:Query 2 tweets in total
INFO:TwitterAPI Logger:Query 1 tweets in total
INFO:TwitterAPI Logger:Query 3 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
<timed exec>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 12 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 1 tweets in total
INFO:TwitterAPI Logger:Query 3 tweets in total
INFO:TwitterAPI Logger:Query 10 tweets in total
<timed exec>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Lo

INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 132 tweets in total
<timed exec>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 62 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 8 tweets in total
INFO:TwitterAPI Logger:Query 53 tweets in total
<timed exec>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 2 tweets in total
INFO:TwitterAPI Logger:Query 2 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI 

INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
<timed exec>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 54 tweets in total
INFO:TwitterAPI Logger:Query 253 tweets in total
INFO:TwitterAPI Logger:Query 132 tweets in total
INFO:TwitterAPI Logger:Query 9 tweets in total
INFO:TwitterAPI Logger:Query 31 tweets in total
INFO:TwitterAPI Logger:Query 1 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 0 tweets in total
INFO:TwitterAPI Logger:Query 11 tw

# 4. Determine the credibility based on the domain name